In [10]:
class Game:
    def __init__(self, N):
        self.available = set()
        for i in range(N):
            for j in range(i+1, N):
                self.available.add((i,j))
        self.game = {i:{} for i in range(N)}
        self.last_player = []
        self.last_x = []
        self.last_y = []


    # checking validity
    def lost(self):
        x,y = self.last_x[-1], self.last_y[-1]
        for j in self.game[x]:
            if self.game[j].get(y, -1) == self.game[j].get(x, -1) == self.last_player[-1]:
                return self.last_player[-1]
        if len(self.available) == 0:
            return True
        return False


    # move addition
    def draw_line(self, x,y, player):
        if x>= len(self.game) or y>= len(self.game) or x in self.game[y]:
            return False
        self.game[x][y] = player
        self.game[y][x] = player
        self.last_player.append(player)
        self.last_x.append(x)
        self.last_y.append(y)
        self.available.remove((min(x,y), max(x,y)))
        return True

    def undo_move(self):
        x = self.last_x.pop()
        y = self.last_y.pop()
        player = self.last_player.pop()
        del self.game[x][y]
        del self.game[y][x]
        self.available.add((min(x,y), max(x,y)))
        return True
        

    # get options
    def get_allowed_lines(self):
        return self.available

In [14]:
def determine_best_next(game: Game, max_min = "MAX", alpha = -float("inf"), beta = float("inf"), depth = 0):
    if game.last_player != []: end = game.lost()
    if game.last_player != [] and end != False:
        if end == True:
            return None, 0.5
        elif max_min == "MIN":
            return None, 0
        else:
            return None, 1
    
    copy = game.available.copy()

    # heuristic
    if depth == 6:
        wins_1 = 0
        wins_2 = 0
        for x,y in copy:
            game.draw_line(x,y, 1)
            if game.lost():
                wins_1 += 1
            game.undo_move()
            game.draw_line(x,y, 2)
            if game.lost():
                wins_2 += 1
            game.undo_move()
        total = wins_1+wins_2
        return None, wins_1/(total) if total >0 else 0.5
            
    
    to_play = None

    if len(game.last_player) == 0 or game.last_player[-1] == 2:
        player = 1
    else:
        player = 2

    if max_min == "MAX":
        res = -1
        for x,y in copy:
            game.draw_line(x,y, player)
            value = determine_best_next(game, "MIN", alpha, beta, depth+1)[1]
            game.undo_move()
            
            if value >= res:
                res = value
                to_play = (x,y)
            alpha = max(alpha, value)

            if alpha > beta:
                break
        return to_play, res
    else:
        res = 2
        for x,y in copy:
            game.draw_line(x,y, player)
            value = determine_best_next(game, "MAX", alpha, beta, depth+1)[1]
            game.undo_move()
            
            if value <= res:
                res = value
                to_play = (x,y)
            beta = min(beta, value)

            if beta < alpha:
                break
            
        return to_play, res


In [ ]:
# play with AI

game = Game(6)

player = 2
while True:
    if player == 2:
        player = 1
    else:
        player = 2
    
    print(f"It is player {player} here")
    print("Available options")
    print(game.get_allowed_lines())
    if player == 1:
        #x = int(input("First: "))
        #y = int(input("Second: "))
        option, val = determine_best_next(game)
        x,y = option
        #while not :
        #    print("Try again")
        #    x = int(input("First: "))
        #    y = int(input("Second: "))
        print(f"You played {(x,y)}")
        game.draw_line(x,y, player)
    else:
        option, val = determine_best_next(game)
        x,y = option
        print(f"AI Played {(x,y)}")
        game.draw_line(x,y, player)
    
    print()

    if game.lost() != False:
        if not isinstance(game.lost(), int):
            print("A Tie!")
        else:
            print(f"Player {game.lost()} lost")
        break

It is player 1 here
Available options
{(0, 1), (2, 4), (1, 2), (0, 4), (3, 4), (1, 5), (0, 3), (1, 4), (2, 3), (0, 2), (4, 5), (0, 5), (2, 5), (1, 3), (3, 5)}
You played (3, 5)

It is player 2 here
Available options
{(0, 1), (3, 4), (1, 5), (0, 4), (2, 4), (1, 2), (0, 2), (0, 5), (1, 3), (2, 5), (0, 3), (1, 4), (2, 3), (4, 5)}
AI Played (1, 3)

It is player 1 here
Available options
{(0, 1), (1, 5), (0, 4), (3, 4), (1, 2), (2, 4), (0, 2), (0, 5), (2, 5), (0, 3), (1, 4), (2, 3), (4, 5)}
You played (2, 5)

It is player 2 here
Available options
{(0, 1), (1, 2), (1, 5), (3, 4), (0, 4), (2, 4), (0, 2), (0, 5), (0, 3), (1, 4), (2, 3), (4, 5)}
AI Played (0, 5)

It is player 1 here
Available options
{(0, 1), (0, 4), (1, 5), (3, 4), (1, 2), (2, 4), (0, 2), (0, 3), (1, 4), (2, 3), (4, 5)}
You played (4, 5)

It is player 2 here
Available options
{(0, 1), (2, 4), (1, 5), (3, 4), (0, 4), (1, 2), (0, 2), (0, 3), (1, 4), (2, 3)}
AI Played (0, 2)

It is player 1 here
Available options
{(0, 1), (1, 5), 